## 必要なライブラリのimport(Notebookを開いたり, Kernel再起動の度に一度だけ実行)

In [1]:
import shutil, os
import pandas as pd
import numpy as np
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import TransformStamped
import copy
from std_msgs.msg import Empty
from triorb_drive_interface.msg import TriorbSetPos3 as TriorbSetPos3Topic
from triorb_drive_interface.msg import TriorbRunResult
from triorb_drive_interface.msg import DriveGains
from scipy.spatial.transform import Rotation as R
rclpy.init()

## 事前定義(Notebookを開いたり, Kernel再起動の度に一度だけ実行)

In [2]:
STD_ACC = 1000
STD_DEC = 1000
GAIN_XY_P = 0.22 # 0.2
GAIN_XY_I = 0.0
GAIN_XY_D = 1e-4 # -5
GAIN_W_P = 0.20
GAIN_W_I = 0.0
GAIN_W_D = 1e-5 # -6

def get_topic_name(topic_name_raw):
    return (os.getenv('ROS_PREFIX', '')+'/'+topic_name_raw).replace("//","/")

def quaternion_to_euler_zyx(q):
    r = R.from_quat([q[0], q[1], q[2], q[3]])
    return r.as_euler('zyx', degrees=False)
    #return r.as_euler('zyx', degrees=True)

class Pose:
    def __init__(self, x,y,z,roll,pitch,yaw):
        self.x = x
        self.y = y
        self.z = z
        self.roll = roll
        self.pitch = pitch
        self.yaw = yaw
        self.deg = -np.rad2deg(yaw)

class LocalizeNode(Node):
    def __init__(self):
        super().__init__('localize_node')
        self.sub_tf = self.create_subscription(TransformStamped, get_topic_name('/vslam/rig_tf'), self.callback_rig_tf, 10)

    def destroy(self):
        _ = self.destroy_subscription(self.sub_tf)
        self.destroy_node()
    
    def callback_rig_tf(self, msg):
        _x, _y, _z = msg.transform.translation.x, msg.transform.translation.y, msg.transform.translation.z
        _yaw, _pitch, _roll  = quaternion_to_euler_zyx([msg.transform.rotation.x, msg.transform.rotation.y, msg.transform.rotation.z, msg.transform.rotation.w])
        self.pose = Pose( _x, _y, _z, _roll, _pitch, _yaw )
        raise KeyboardInterrupt


class NavigateNode(Node):
    def __init__(self):
        super().__init__('sample_abs_pos')
        self.pub_gain = self.create_publisher(DriveGains, get_topic_name("/setting/drive/gains"), 1)
        self.pub_set_pos = self.create_publisher(TriorbSetPos3Topic, get_topic_name("/drive/set_pos"), 1)
        self.pub_stop = self.create_publisher(Empty, get_topic_name("/drive/stop"), 10)
        self.subs = [self.create_subscription(TriorbRunResult, get_topic_name('/drive/result'), self.callback_result, 1)]

    def run_pos(self, x,y,deg, vxy=0.2, vw=0.5, tx=0.1, ty=0.1, tr=5.0, force=1):
        request = TriorbSetPos3Topic()
        request.pos.position.x = x
        request.pos.position.y = y
        request.pos.position.deg = deg
        request.pos.speed.xy = vxy
        request.pos.speed.w = vw
        request.pos.speed.acc = STD_ACC
        request.pos.speed.dec = STD_DEC
        request.setting.tx = tx
        request.setting.ty = ty
        request.setting.tr = tr
        request.setting.force = force
        self.pub_set_pos.publish( request )
        raise KeyboardInterrupt
            
    def set_gain(self):
        msg = DriveGains()
        msg.xy_p = GAIN_XY_P
        msg.xy_i = GAIN_XY_I
        msg.xy_d = GAIN_XY_D
        msg.w_p = GAIN_W_P
        msg.w_i = GAIN_W_I
        msg.w_d = GAIN_W_D
        self.pub_gain.publish(msg)
        raise KeyboardInterrupt

    def callback_result(self, msg):
        print("Receive results")
        print(str(msg))
        raise KeyboardInterrupt

    def stop(self):
        self.pub_stop.publish(Empty())
        raise KeyboardInterrupt
    
    def destroy(self):
        self.pub_stop.publish(Empty())
        _ = [self.destroy_subscription(_sub) for _sub in self.subs]
        self.destroy_node()


localize_node = LocalizeNode()
nav_node = NavigateNode()

def get_vlam_pose():
    try:
        rclpy.spin(localize_node)
    except KeyboardInterrupt:
        pass
    #node.destroy()
    return localize_node.pose

def set_pos_absolute(x,y,deg, vxy=0.2, vw=0.5, tx=0.1, ty=0.1, tr=5.0, force=1):
    try:
        nav_node.set_gain()
        rclpy.spin(nav_node)
    except KeyboardInterrupt:
        pass

    try:
        nav_node.run_pos(x,y,deg, vxy, vw, tx, ty, tr, force)
        rclpy.spin(nav_node)
    except KeyboardInterrupt:
        pass


def join():
    try:
        rclpy.spin(nav_node)
    except KeyboardInterrupt:
        pass

def force_stop():
    try:
        nav_node.stop()
        rclpy.spin(nav_node)
    except KeyboardInterrupt:
        pass


### 現在位置確認
Visual SLAMにより推定された姿勢を取得します.  単位はx[<span style="color:#FF0000">vm</span>], y[<span style="color:#FF0000">vm</span>], z[<span style="color:#FF0000">vm</span>], roll[rad], pitch[rad], yaw[rad], deg[°]です.<br>
実際に使用する値は, x,y,deg の3つです. degはyawの値を[°]に変換したものです.<br>
x,y,zの単位が[vm]となっているのは, スケールvが地図ごとに0.25~2.0程度変動するためです. vは地図作成終了まで不定です.<br>
また, 現在位置は停止状態でも光の加減によって値が微小に変化します.  

In [ ]:
pose = get_vlam_pose()
print("x :", pose.x)
print("y :", pose.y)
print("z :", pose.z)
print("roll :", pose.roll)
print("pitch:", pose.pitch)
print("yaw  :", pose.yaw)
print("x,y,w = ({}, {}, {})".format(pose.x, pose.y, pose.deg) )

## 絶対位値指令
地図上のx,y,deg座標に移動します. 単位は, x[vm], y[vm], deg[°], vxy[m/s], vw[rad/s] です. <br>
force=1の場合, 位置フィードバック制御を行います. このとき, 目標姿勢と現在姿勢の差がtx[vm], ty[vm], tr[°]になるか, 自己位置を見失うまで移動し続けます. <br>
force=0を指定すると位置フィードバック制御を行わず, スケールを考慮しないため, 指示と異なる姿勢で止まることが多くなります.

In [ ]:
# -x方向（左）に 0.2[m/s] で移動. 停止精度をx=0.01[vm], y=0.01[vm], w=3[°] とする.
pose = get_vlam_pose()
print("現在姿勢 : ", pose.x,pose.y,pose.deg)
# 目標姿勢
x   = pose.x-0.2
y   = pose.y
deg = pose.deg
# 移動速度
vxy=0.2
vw=0.5
# 停止精度
tx=0.01
ty=0.01
tr=3.0
# 位置フィードバック
force=1

# 移動指示
set_pos_absolute(x,y,deg, vxy, vw, tx, ty, tr, force)

In [ ]:
pose = get_vlam_pose()
print("移動後姿勢 : ", pose.x, pose.y, pose.deg)

## 移動の中断

In [20]:
# 移動を止めたい場合に実行
force_stop()